In [1]:
from selenium import webdriver # 자바스크립트로 전송 되는 데이터는 url - html에 안붙어있습니다.. 페이지를 열어줘야 데이터가 와요!
import lxml
import time
import requests
from selenium.webdriver.common.keys import Keys

In [2]:
path = 'chromedriver.exe' # 크롬드라이브 경로 지정  ## 맥과 리눅스는 exe.를 붙이지 않습니다.

driver = webdriver.Chrome(path)

driver.implicitly_wait(3) # 기다려줍시다

In [3]:
driver.get("http://www.kobis.or.kr/kobis/business/main/main.do")

driver.implicitly_wait(3)

In [4]:
daily_boxoffice_html = driver.page_source

In [5]:
from bs4 import BeautifulSoup as bs
parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'html.parser')

In [6]:
daily_boxoffice = parsed_daily_boxoffice_html.find("div", {"id":"dailyBoxOffice"})

print(daily_boxoffice)

<div class="mainmovie3" id="dailyBoxOffice">
<div class="maindata" id="maindata1"><table cellpadding="0" cellspacing="0" class="mainover1" style="width: 310px;"><tbody><tr onmouseout="MM_showHideLayers('poster1','','mainhide','maindata1','','mainhide');fnPosterNm('1');" onmouseover="MM_showHideLayers('poster1','','mainshow','maindata1','','mainshow');fnMovieNm('1','샤잠!');"><td class="maindata1"><img alt="1위 " src="/kobis/web/comm/images/main/nember_A01_on.png"/></td><td class="maindata2"><a href="#none" onclick="mstView('movie','20196244')" style="color:#bdea0a;">샤잠!</a></td><td class="maindata3">1,442,542,799원</td><td class="maindata4">159,709명</td><td><div><ul><li class="maindata5"><img alt="유지" class="mainarrow1" src="/kobis/web/comm/images/main/arrow3.png"/></li><li class="maindata6">0</li></ul></div></td></tr><tr onmouseout="MM_showHideLayers('poster1','','mainhide','maindata1','','mainhide');fnPosterNm('1');" onmouseover="MM_showHideLayers('poster1','','mainshow','maindata1','','

In [7]:
movie_list = daily_boxoffice.find_all("div", {"class":"maindata"})
for movie in movie_list:
    print("movie : %s" % movie.text[:-1])

movie : 샤잠!1,442,542,799원159,709명
movie : 생일983,373,413원113,342명
movie : 돈897,833,456원99,665명
movie : 어스866,242,771원95,398명
movie : 장난스런 키스315,510,970원37,263명
movie : 덤보219,028,530원27,043명
movie : 캡틴 마블238,635,000원26,649명
movie : 로망59,982,050원7,412명
movie : 고릴라와 슈퍼레이스14,320,870원1,884명1
movie : 콜레트15,751,880원1,809명


In [8]:
driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do?loadEnd=0&searchType=search&sSearchFrom=2019-03-17&sSearchTo=2019-03-24&sMultiMovieYn=&sRepNationCd=&sWideAreaCd=")

driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

driver.find_element_by_id("btn_0").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(2)

driver.find_element_by_id("btn_0").click()

driver.implicitly_wait(2)

driver.find_element_by_id("btn_0").click()

driver.implicitly_wait(2)

driver.find_element_by_id("btn_0").click()

daily_boxoffice_html = driver.page_source

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'html.parser')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_0"})

movie_list = daily_boxoffice.find_all("tr")

import pandas as pd

daily_movies = {'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
 
daily_boxoffice1 = pd.DataFrame(daily_movies) 

daily_boxoffice1

,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
0,돈,동일,2019-03-20,"3,753,473,160",59.7%,"416,173","1,536,401","1,431","7,433"
1,캡틴 마블,동일,2019-03-06,"1,275,765,220",20.3%,"142,380","5,232,807",947,"3,768"
2,악질경찰,동일,2019-03-20,"427,941,590",6.8%,"46,589","189,604",718,"2,513"
3,우상,동일,2019-03-20,"254,358,090",4.0%,"28,126","137,880",659,"1,884"
4,이스케이프 룸,동일,2019-03-14,"214,363,930",3.4%,"23,494","544,462",491,"1,060"
5,정글북: 마법 원정대,동일,2019-03-21,"90,106,890",1.4%,"11,520","22,534",372,495
6,페이트 스테이 나이트 헤븐즈필 제2장 로스트 버터플라이,동일,2019-03-21,"48,092,180",0.8%,"5,292","24,340",94,152
7,라스트 미션,1상승,2019-03-14,"27,805,210",0.4%,"3,345","93,476",101,169
8,그린 북,1하락,2019-01-09,"23,715,060",0.4%,"3,302","409,871",62,99
9,장난스런 키스,동일,2019-03-27,"30,718,400",0.5%,"3,221","8,588",8,13


In [9]:
def change_rank(rank):
    if rank == "동일":
        rank = 0
    elif rank[-2:] == "하락":
        rank = int(rank[:-2]) * -1
    elif rank[-2:] == "상승":
        rank = int(rank[:-2]) * 1
    return rank

daily_boxoffice1['순위 변동'] = daily_boxoffice1['순위 변동'].map(lambda x: change_rank(x))

        
daily_boxoffice1

,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
0,돈,0,2019-03-20,"3,753,473,160",59.7%,"416,173","1,536,401","1,431","7,433"
1,캡틴 마블,0,2019-03-06,"1,275,765,220",20.3%,"142,380","5,232,807",947,"3,768"
2,악질경찰,0,2019-03-20,"427,941,590",6.8%,"46,589","189,604",718,"2,513"
3,우상,0,2019-03-20,"254,358,090",4.0%,"28,126","137,880",659,"1,884"
4,이스케이프 룸,0,2019-03-14,"214,363,930",3.4%,"23,494","544,462",491,"1,060"
5,정글북: 마법 원정대,0,2019-03-21,"90,106,890",1.4%,"11,520","22,534",372,495
6,페이트 스테이 나이트 헤븐즈필 제2장 로스트 버터플라이,0,2019-03-21,"48,092,180",0.8%,"5,292","24,340",94,152
7,라스트 미션,1,2019-03-14,"27,805,210",0.4%,"3,345","93,476",101,169
8,그린 북,-1,2019-01-09,"23,715,060",0.4%,"3,302","409,871",62,99
9,장난스런 키스,0,2019-03-27,"30,718,400",0.5%,"3,221","8,588",8,13


In [10]:
driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do?loadEnd=0&searchType=search&sSearchFrom=2019-03-17&sSearchTo=2019-03-24&sMultiMovieYn=&sRepNationCd=&sWideAreaCd=")

driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

driver.find_element_by_id("btn_118").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(2)

driver.find_element_by_id("btn_118").click()

driver.implicitly_wait(2)

driver.find_element_by_id("btn_118").click()

driver.implicitly_wait(2)

driver.find_element_by_id("btn_118").click()

daily_boxoffice_html = driver.page_source

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'html.parser')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_118"})

movie_list = daily_boxoffice.find_all("tr")

import pandas as pd

daily_movies = {'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
 
daily_boxoffice2 = pd.DataFrame(daily_movies) 

daily_boxoffice2['순위 변동'] = daily_boxoffice2['순위 변동'].map(lambda x: change_rank(x))

daily_boxoffice2

,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
0,돈,0,2019-03-20,"4,347,856,286",59.0%,"480,713","1,120,228","1,410","7,486"
1,캡틴 마블,0,2019-03-06,"1,621,661,501",22.0%,"178,487","5,090,427",938,"3,821"
2,악질경찰,0,2019-03-20,"459,773,175",6.2%,"50,107","143,015",731,"2,676"
3,우상,0,2019-03-20,"294,901,025",4.0%,"32,698","109,754",675,"2,052"
4,이스케이프 룸,0,2019-03-14,"253,327,669",3.4%,"27,842","520,968",496,"1,067"
5,정글북: 마법 원정대,8,2019-03-21,"75,837,650",1.0%,"9,775","11,014",354,478
6,페이트 스테이 나이트 헤븐즈필 제2장 로스트 버터플라이,-1,2019-03-21,"67,392,050",0.9%,"7,306","19,048",90,147
7,그린 북,2,2019-01-09,"27,736,410",0.4%,"3,802","406,569",57,89
8,라스트 미션,-1,2019-03-14,"32,349,160",0.4%,"3,801","90,131",103,172
9,장난스런 키스,-3,2019-03-27,"27,681,850",0.4%,"2,970","5,367",16,17


In [11]:
driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do?loadEnd=0&searchType=search&sSearchFrom=2019-03-17&sSearchTo=2019-03-24&sMultiMovieYn=&sRepNationCd=&sWideAreaCd=")

driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

driver.find_element_by_id("btn_232").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(2)

driver.find_element_by_id("btn_232").click()

driver.implicitly_wait(2)

driver.find_element_by_id("btn_232").click()

driver.implicitly_wait(2)

driver.find_element_by_id("btn_232").click()

daily_boxoffice_html = driver.page_source

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'html.parser')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_232"})

movie_list = daily_boxoffice.find_all("tr")

import pandas as pd

daily_movies = {'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
 
daily_boxoffice3 = pd.DataFrame(daily_movies) 

daily_boxoffice3['순위 변동'] = daily_boxoffice3['순위 변동'].map(lambda x: change_rank(x))

daily_boxoffice3

,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
0,돈,0,2019-03-20,"1,941,016,383",58.9%,"214,510","639,515","1,268","6,372"
1,캡틴 마블,0,2019-03-06,"609,926,536",18.5%,"65,373","4,911,940",812,"3,252"
2,악질경찰,0,2019-03-20,"247,429,553",7.5%,"27,478","92,908",772,"2,910"
3,우상,0,2019-03-20,"176,216,937",5.3%,"19,826","77,056",797,"2,566"
4,이스케이프 룸,0,2019-03-14,"116,031,904",3.5%,"12,564","493,126",447,"1,051"
5,페이트 스테이 나이트 헤븐즈필 제2장 로스트 버터플라이,0,2019-03-21,"42,813,010",1.3%,"4,447","11,742",80,129
6,장난스런 키스,0,2019-03-27,"22,296,900",0.7%,"2,100","2,397",10,10
7,라스트 미션,1,2019-03-14,"16,801,460",0.5%,"2,032","86,330",114,189
8,양지의 그녀,-2,2019-03-21,"17,272,590",0.5%,"1,872","5,425",133,231
9,그린 북,3,2019-01-09,"12,020,230",0.4%,"1,644","402,767",56,86


In [12]:
driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do?loadEnd=0&searchType=search&sSearchFrom=2019-03-17&sSearchTo=2019-03-24&sMultiMovieYn=&sRepNationCd=&sWideAreaCd=")

driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

driver.find_element_by_id("btn_341").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(2)

driver.find_element_by_id("btn_341").click()

driver.implicitly_wait(2)

driver.find_element_by_id("btn_341").click()

driver.implicitly_wait(2)

driver.find_element_by_id("btn_341").click()

daily_boxoffice_html = driver.page_source

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'html.parser')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_341"})

movie_list = daily_boxoffice.find_all("tr")

import pandas as pd

daily_movies = {'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
 
daily_boxoffice4 = pd.DataFrame(daily_movies) 

daily_boxoffice4['순위 변동'] = daily_boxoffice4['순위 변동'].map(lambda x: change_rank(x))
daily_boxoffice4

,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
0,돈,0,2019-03-20,"1,407,813,290",56.2%,"170,825","425,005","1,166","5,802"
1,캡틴 마블,0,2019-03-06,"388,328,160",15.5%,"45,256","4,846,567",738,"2,923"
2,악질경찰,0,2019-03-20,"237,173,250",9.5%,"29,275","65,430",807,"3,106"
3,우상,0,2019-03-20,"165,600,800",6.6%,"20,549","57,230",849,"2,864"
4,이스케이프 룸,0,2019-03-14,"82,366,980",3.3%,"9,784","480,562",456,"1,050"
5,페이트 스테이 나이트 헤븐즈필 제2장 로스트 버터플라이,New,2019-03-21,"49,977,700",2.0%,"5,963","7,295",79,131
6,양지의 그녀,4,2019-03-21,"19,996,470",0.8%,"2,423","3,553",133,231
7,항거:유관순 이야기,1,2019-02-27,"13,573,330",0.5%,"2,021","1,141,084",69,87
8,라스트 미션,-2,2019-03-14,"14,321,100",0.6%,"1,914","84,298",111,187
9,생일,6,2019-04-03,"16,677,000",0.7%,"1,853","6,546",9,9


In [13]:
driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do?loadEnd=0&searchType=search&sSearchFrom=2019-03-17&sSearchTo=2019-03-24&sMultiMovieYn=&sRepNationCd=&sWideAreaCd=")

driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

driver.find_element_by_id("btn_431").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(2)

driver.find_element_by_id("btn_431").click()

driver.implicitly_wait(2)

driver.find_element_by_id("btn_431").click()

driver.implicitly_wait(2)

driver.find_element_by_id("btn_431").click()

daily_boxoffice_html = driver.page_source

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'html.parser')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_431"})

movie_list = daily_boxoffice.find_all("tr")

import pandas as pd

daily_movies = {'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
 
daily_boxoffice5 = pd.DataFrame(daily_movies) 

daily_boxoffice5['순위 변동'] = daily_boxoffice5['순위 변동'].map(lambda x: change_rank(x))

daily_boxoffice5

,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
0,돈,3,2019-03-20,"1,369,138,680",54.4%,"165,833","254,180","1,146","5,258"
1,캡틴 마블,-1,2019-03-06,"395,671,940",15.7%,"46,019","4,801,311",729,"2,840"
2,악질경찰,9,2019-03-20,"270,162,120",10.7%,"33,072","36,155",833,"3,059"
3,우상,4,2019-03-20,"222,873,850",8.9%,"27,175","36,681",900,"2,892"
4,이스케이프 룸,-3,2019-03-14,"101,006,850",4.0%,"12,025","470,778",513,"1,196"
5,1919 유관순,3,2019-03-14,"17,036,400",0.7%,"2,147","22,696",122,130
6,라스트 미션,-4,2019-03-14,"15,043,290",0.6%,"2,010","82,384",177,248
7,그린 북,3,2019-01-09,"9,440,050",0.4%,"1,378","399,920",48,86
8,항거:유관순 이야기,-3,2019-02-27,"7,866,480",0.3%,"1,243","1,139,063",106,119
9,보헤미안 랩소디,0,2018-10-31,"10,968,000",0.4%,"1,218","9,945,756",6,6


In [14]:
driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do?loadEnd=0&searchType=search&sSearchFrom=2019-03-17&sSearchTo=2019-03-24&sMultiMovieYn=&sRepNationCd=&sWideAreaCd=")

driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

driver.find_element_by_id("btn_526").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(2)

driver.find_element_by_id("btn_526").click()

driver.implicitly_wait(2)

driver.find_element_by_id("btn_526").click()

driver.implicitly_wait(2)

driver.find_element_by_id("btn_526").click()

daily_boxoffice_html = driver.page_source

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'html.parser')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_526"})

movie_list = daily_boxoffice.find_all("tr")

import pandas as pd

daily_movies = {'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
 
daily_boxoffice6 = pd.DataFrame(daily_movies) 

daily_boxoffice6['순위 변동'] = daily_boxoffice6['순위 변동'].map(lambda x: change_rank(x))
daily_boxoffice6

,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
0,캡틴 마블,0,2019-03-06,"609,359,330",47.5%,"72,224","4,755,292","1,599","7,813"
1,이스케이프 룸,0,2019-03-14,"271,195,500",21.2%,"32,292","458,753",756,"3,251"
2,라스트 미션,0,2019-03-14,"60,288,910",4.7%,"7,829","80,374",455,"1,101"
3,돈,0,2019-03-20,"54,865,600",4.3%,"6,190","88,347",35,35
4,극한직업,0,2019-01-23,"37,019,810",2.9%,"4,813","16,252,582",340,579
5,항거:유관순 이야기,0,2019-02-27,"32,572,680",2.5%,"4,435","1,137,820",399,751
6,증인,0,2019-02-13,"30,369,050",2.4%,"4,121","2,527,267",367,650
7,우상,New,2019-03-20,"32,345,400",2.5%,"3,432","9,506",28,28
8,1919 유관순,2,2019-03-14,"17,818,580",1.4%,"2,197","20,549",218,341
9,사바하,-1,2019-02-20,"17,355,270",1.4%,"2,149","2,394,285",272,427


In [15]:
driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do?loadEnd=0&searchType=search&sSearchFrom=2019-03-17&sSearchTo=2019-03-24&sMultiMovieYn=&sRepNationCd=&sWideAreaCd=")

driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

driver.find_element_by_id("btn_622").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(2)

driver.find_element_by_id("btn_622").click()

driver.implicitly_wait(2)

driver.find_element_by_id("btn_622").click()

driver.implicitly_wait(2)

driver.find_element_by_id("btn_622").click()

daily_boxoffice_html = driver.page_source

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'html.parser')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_622"})

movie_list = daily_boxoffice.find_all("tr")

import pandas as pd

daily_movies = {'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
 
daily_boxoffice7 = pd.DataFrame(daily_movies) 

daily_boxoffice7['순위 변동'] = daily_boxoffice7['순위 변동'].map(lambda x: change_rank(x))
daily_boxoffice7

,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
0,캡틴 마블,0,2019-03-06,"699,257,770",51.2%,"82,944","4,683,068","1,592","8,043"
1,이스케이프 룸,0,2019-03-14,"310,772,720",22.7%,"36,891","426,461",751,"3,359"
2,라스트 미션,1,2019-03-14,"63,227,320",4.6%,"8,243","72,545",460,"1,148"
3,돈,-1,2019-03-20,"38,703,000",2.8%,"6,693","82,157",17,23
4,극한직업,1,2019-01-23,"34,344,980",2.5%,"4,531","16,247,769",339,597
5,항거:유관순 이야기,-1,2019-02-27,"33,643,970",2.5%,"4,489","1,133,385",406,769
6,증인,0,2019-02-13,"30,810,100",2.3%,"3,996","2,523,146",373,657
7,생일,New,2019-04-03,"25,587,000",1.9%,"2,843","3,862",12,12
8,사바하,0,2019-02-20,"19,580,110",1.4%,"2,376","2,392,136",279,439
9,그린 북,0,2019-01-09,"14,574,860",1.1%,"2,074","396,454",73,146


In [31]:
import sqlite3

con = sqlite3.connect("daily_boxoffice.db")

con.execute("CREATE TABLE IF NOT EXISTS boxofficeMar24(title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")
con.execute("CREATE TABLE IF NOT EXISTS boxofficeMar23(title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")
con.execute("CREATE TABLE IF NOT EXISTS boxofficeMar22(title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")
con.execute("CREATE TABLE IF NOT EXISTS boxofficeMar21(title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")
con.execute("CREATE TABLE IF NOT EXISTS boxofficeMar20(title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")
con.execute("CREATE TABLE IF NOT EXISTS boxofficeMar19(title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")
con.execute("CREATE TABLE IF NOT EXISTS boxofficeMar18(title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")
daily_boxoffice1.to_sql("boxofficeMar24", con, chunksize=1000, if_exists='replace')
daily_boxoffice2.to_sql("boxofficeMar23", con, chunksize=1000, if_exists='replace')
daily_boxoffice3.to_sql("boxofficeMar22", con, chunksize=1000, if_exists='replace')
daily_boxoffice4.to_sql("boxofficeMar21", con, chunksize=1000, if_exists='replace')
daily_boxoffice5.to_sql("boxofficeMar20", con, chunksize=1000, if_exists='replace')
daily_boxoffice6.to_sql("boxofficeMar19", con, chunksize=1000, if_exists='replace')
daily_boxoffice7.to_sql("boxofficeMar18", con, chunksize=1000, if_exists='replace')

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


In [32]:
con.commit()

con.close()

In [33]:
import sqlite3
import pandas as pd

con = sqlite3.connect("daily_boxoffice.db")

cursor = con.cursor()

In [35]:
cursor.execute("SELECT * FROM boxofficeMar23")

data = cursor.fetchall()
df = pd.read_sql("SELECT * FROM boxofficeMar23", con, index_col='index')

df

,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
index,,,,,,,,,
0,돈,0,2019-03-20,"4,347,856,286",59.0%,"480,713","1,120,228","1,410","7,486"
1,캡틴 마블,0,2019-03-06,"1,621,661,501",22.0%,"178,487","5,090,427",938,"3,821"
2,악질경찰,0,2019-03-20,"459,773,175",6.2%,"50,107","143,015",731,"2,676"
3,우상,0,2019-03-20,"294,901,025",4.0%,"32,698","109,754",675,"2,052"
4,이스케이프 룸,0,2019-03-14,"253,327,669",3.4%,"27,842","520,968",496,"1,067"
5,정글북: 마법 원정대,8,2019-03-21,"75,837,650",1.0%,"9,775","11,014",354,478
6,페이트 스테이 나이트 헤븐즈필 제2장 로스트 버터플라이,-1,2019-03-21,"67,392,050",0.9%,"7,306","19,048",90,147
7,그린 북,2,2019-01-09,"27,736,410",0.4%,"3,802","406,569",57,89
8,라스트 미션,-1,2019-03-14,"32,349,160",0.4%,"3,801","90,131",103,172
